<a href="https://colab.research.google.com/github/lastinm/ml_hw_notebooks/blob/main/%D0%A5%D1%80%D0%B0%D0%BD%D0%B5%D0%BD%D0%B8%D0%B5%2C_%D0%B8%D0%B7%D0%B2%D0%BB%D0%B5%D1%87%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B8_%D0%BE%D0%B1%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D0%B0_%D0%91%D0%94_%D0%9B%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%BE%D1%80%D0%BD%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%E2%84%965.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание 1. Structured Streaming из набора файлов

In [16]:
from pyspark.sql import SparkSession

spark = SparkSession \
  .builder \
  .appName("Streaming 1") \
  .config("spark.sql.streaming.schemaInference", "true") \
  .getOrCreate()

1. Подготовить данные, имеющиеся во вложении.
2. Открыть на основе исходных json файлов потоковый датафрейм.

In [17]:
df = spark.readStream \
  .format("json") \
  .option("path", "/content/sample_data") \
  .option("maxFilesPerTrigger", 1) \
  .load()

3. Написать преобразование исходного датафрейма, для получения плоской (flat) структуры.

In [18]:
df2 = df.selectExpr("InvoiceNumber",
                    "CreatedTime",
                    "StoreID",
                    "PosID",
                    "CashierID",
                    "CustomerType",
                    "CustomerCardNo",
                    "TotalAmount",
                    "NumberOfItems",
                    "PaymentMethod",
                    "TaxableAmount",
                    "explode(InvoiceLineItems) as Item")

In [19]:
from pyspark.sql.functions import expr

# [{"ItemCode":"458","ItemDescription":"Wine glass","ItemPrice":1644.0,"ItemQty":2,"TotalValue":3288.0},{"ItemCode":"283","ItemDescription":"Portable Lamps","ItemPrice":2236.0,"ItemQty":1,"TotalValue":2236.0},{"ItemCode":"498","ItemDescription":"Carving knifes","ItemPrice":1424.0,"ItemQty":2,"TotalValue":2848.0},{"ItemCode":"523","ItemDescription":"Oil-lamp clock","ItemPrice":1371.0,"ItemQty":2,"TotalValue":2742.0}]
df3 = df2 \
  .withColumn("ItemCode", expr("Item.ItemCode")) \
  .withColumn("ItemDescription", expr("Item.ItemDescription")) \
  .withColumn("ItemPrice", expr("Item.ItemPrice")) \
  .withColumn("ItemQty", expr("Item.ItemQty")) \
  .withColumn("TotalValue", expr("Item.TotalValue")) \
  .drop("Item")

4. Сохранять получаемые каждые X секунд исходные файлы в json.

In [21]:
query = df3 \
  .writeStream \
  .format("json") \
  .option("path", "/content/output") \
  .option("checkpointLocation", "/content/checkpoint") \
  .trigger(processingTime="10 seconds") \
  .start()

In [25]:
query.lastProgress

{'id': 'b5c88650-07d1-4a2f-80bc-4787b726564a',
 'runId': '36e98a25-8bab-4937-b15d-d2283405e630',
 'name': None,
 'timestamp': '2025-01-04T18:53:40.000Z',
 'batchId': 9,
 'numInputRows': 0,
 'inputRowsPerSecond': 0.0,
 'processedRowsPerSecond': 0.0,
 'durationMs': {'latestOffset': 4, 'triggerExecution': 6},
 'stateOperators': [],
 'sources': [{'description': 'FileStreamSource[file:/content/sample_data]',
   'startOffset': {'logOffset': 8},
   'endOffset': {'logOffset': 8},
   'latestOffset': None,
   'numInputRows': 0,
   'inputRowsPerSecond': 0.0,
   'processedRowsPerSecond': 0.0}],
 'sink': {'description': 'FileSink[/content/output]', 'numOutputRows': -1}}

In [6]:
spark.stop()

{"InvoiceNumber":"51402977","CreatedTime":1595688900348,"StoreID":"STR7188","PosID":"POS956","CashierID":"OAS134","CustomerType":"PRIME","CustomerCardNo":"4629185211","TotalAmount":11114.0,"NumberOfItems":4,"PaymentMethod":"CARD","TaxableAmount":11114.0,"CGST":277.85,"SGST":277.85,"CESS":13.8925,"DeliveryType":"TAKEAWAY","InvoiceLineItems":[{"ItemCode":"458","ItemDescription":"Wine glass","ItemPrice":1644.0,"ItemQty":2,"TotalValue":3288.0},{"ItemCode":"283","ItemDescription":"Portable Lamps","ItemPrice":2236.0,"ItemQty":1,"TotalValue":2236.0},{"ItemCode":"498","ItemDescription":"Carving knifes","ItemPrice":1424.0,"ItemQty":2,"TotalValue":2848.0},{"ItemCode":"523","ItemDescription":"Oil-lamp clock","ItemPrice":1371.0,"ItemQty":2,"TotalValue":2742.0}]}